# Distilling Vision Transformers

**Author:** [Sayak Paul](https://twitter.com/RisingSayak)<br>
**Date created:** 2022/04/05<br>
**Last modified:** 2022/04/08<br>
**Description:** Distillation of Vision Transformers through attention.<br>
**Translation:** [Junghyun Park](https://github.com/parkjh688)

## 들어가며

*Vision Transformers* (ViT) 논문 ([Dosovitskiy et al.](https://arxiv.org/abs/2010.11929))에서 저자들은 ViTs가 Convolutional Neural Networks(CNNs)에 비하는 성능을 내려면 더 큰 데이터셋으로 pre-train을 진행해야한다고 했습니다. 그리고 그 데이터셋은 크면 클수록 좋습니다. ViT 아키텍쳐엔 CNNs과는 달리 inductive biases와 지역성(locality)에 대한 레이어가 없기 때문입니다. 이후의 논문 [Steiner et al.](https://arxiv.org/abs/2106.10270)에서 저자들은 더 강력한 정규화(regularization)와 더 긴 학습으로 ViT의 성능을 실질적으로 향상시키는 것이 가능하다는 것을 보여줍니다.

많은 그룹이 ViT 학습시에 data-intensiveness 문제를 다루는 다양한 방법을 제안했습니다. 그 방법 중 하나는 *Data-efficient image Transformers*, (DeiT) [Touvron et al.](https://arxiv.org/abs/2012.12877)에서 제안했으며, 저자들은 vision transformer 모델에 특화된 distillation 방식을 소개했습니다. DeiT는 더 큰 데이터 셋을 사용하지 않고도 ViT를 잘 학습할 수 있다는 것을 보여줬습니다.

이 튜토리얼에서 우리는 DeiT 논문에서 제안한 distillation 레시피를 구현할 것이고, 구현을 위해 ViT 아키텍처를 약간 수정하고 distillation 레시피를 구현하기 위한 custom training loop를 만들 것입니다.

튜토리얼을 실행하려면 다음 명령을 사용하여 설치할 수 있는 TensorFlow Addons을 설치해야합니다.
```
pip install tensorflow-addons
```

이 튜토리얼을 쉽게 이해하려면 일단 ViT와 knowledge distillation이 어떻게 동작하는지 아는 것이 좋습니다. 아래의 리소스가 당신을 도와줄 것입니다:

* [ViT on keras.io](https://keras.io/examples/vision/image_classification_with_vision_transformer)
* [Knowledge distillation on keras.io](https://keras.io/examples/vision/knowledge_distillation/)

## Imports

In [ ]:
# colab에는 tensorflow-addons 설치 되어있지 않으므로 설치
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [ ]:
from typing import List

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

tfds.disable_progress_bar()
tf.keras.utils.set_random_seed(42)

## Constants

In [ ]:
# Model
MODEL_TYPE = "deit_distilled_tiny_patch16_224"
RESOLUTION = 224
PATCH_SIZE = 16
NUM_PATCHES = (RESOLUTION // PATCH_SIZE) ** 2
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 192
NUM_HEADS = 3
NUM_LAYERS = 12
MLP_UNITS = [
    PROJECTION_DIM * 4,
    PROJECTION_DIM,
]
DROPOUT_RATE = 0.0
DROP_PATH_RATE = 0.1

# Training
NUM_EPOCHS = 20
BASE_LR = 0.0005
WEIGHT_DECAY = 0.0001

# Data
BATCH_SIZE = 256
AUTO = tf.data.AUTOTUNE
NUM_CLASSES = 5

**DROPOUT_RATE**가 0.0으로 설정되었다는 것을 눈치채셨을 것 같습니다. 이 예제에 사용된 작은(smaller) 모델의 경우 필요하지 않지만 큰(bigger) 모델의 경우 드롭아웃을 사용합니다.

## `tf_flowers` 데이터셋 그리고 전처리 유틸리티 가져오기

MixUp ([Zhang et al.](https://arxiv.org/abs/1710.09412))과 RandAugment ([Cubuk et al.](https://arxiv.org/abs/1909.13719))의 저자들은 다양한 augmentation 기법을 사용했습니다. </br>
그러나 튜토리얼의 단순화를 위하여 우리는 그 부분은 사용하지 않을 것입니다

In [ ]:
def preprocess_dataset(is_training=True):
    def fn(image, label):
        if is_training:
            # 더 큰 공간 해상도로 크기를 리사이즈하고 랜덤 샘플링
            # 크롭
            image = tf.image.resize(image, (RESOLUTION + 20, RESOLUTION + 20))
            image = tf.image.random_crop(image, (RESOLUTION, RESOLUTION, 3))
            image = tf.image.random_flip_left_right(image)
        else:
            image = tf.image.resize(image, (RESOLUTION, RESOLUTION))
        label = tf.one_hot(label, depth=NUM_CLASSES)
        return image, label

    return fn


def prepare_dataset(dataset, is_training=True):
    if is_training:
        dataset = dataset.shuffle(BATCH_SIZE * 10)
    dataset = dataset.map(preprocess_dataset(is_training), num_parallel_calls=AUTO)
    return dataset.batch(BATCH_SIZE).prefetch(AUTO)


train_dataset, val_dataset = tfds.load(
    "tf_flowers", split=["train[:90%]", "train[90%:]"], as_supervised=True
)
num_train = train_dataset.cardinality()
num_val = val_dataset.cardinality()
print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")

train_dataset = prepare_dataset(train_dataset, is_training=True)
val_dataset = prepare_dataset(val_dataset, is_training=False)

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Number of training examples: 3303
Number of validation examples: 367


### ViT 변형인 DeiT 구현하기

DeiT는 ViT의 확장 버전이기 때문에 먼저 ViT를 구현한 다음 DeiT의 구성 요소를 추가하는 것이 알맞을 것입니다. </br>

우선, 우리는 DeiT가 정규화(regularization)를 위해 사용하는 Stochastic Depth ([Huang et al.](https://arxiv.org/abs/1603.09382)) 레이어를 구현할겁니다.

In [ ]:
# Referred from: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=True):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x


이제, MLP와 Transformer 블록을 구현해봅시다.

In [ ]:

def mlp(x, dropout_rate: float, hidden_units: List):
    """FFN for a Transformer block."""
    # hidden_units 만큼 반복하면서 Dense 레이어와 Dropout 레이어 추가
    for (idx, units) in enumerate(hidden_units):
        x = layers.Dense(
            units,
            activation=tf.nn.gelu if idx == 0 else None,
        )(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer(drop_prob: float, name: str) -> keras.Model:
    """Transformer block with pre-norm."""
    num_patches = NUM_PATCHES + 2 if "distilled" in MODEL_TYPE else NUM_PATCHES + 1
    encoded_patches = layers.Input((num_patches, PROJECTION_DIM))

    # Layer normalization 1.
    x1 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(encoded_patches)

    # Multi Head Self Attention layer 1.
    attention_output = layers.MultiHeadAttention(
        num_heads=NUM_HEADS,
        key_dim=PROJECTION_DIM,
        dropout=DROPOUT_RATE,
    )(x1, x1)
    attention_output = (
        StochasticDepth(drop_prob)(attention_output) if drop_prob else attention_output
    )

    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])

    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(x2)

    # MLP layer 1.
    x4 = mlp(x3, hidden_units=MLP_UNITS, dropout_rate=DROPOUT_RATE)
    x4 = StochasticDepth(drop_prob)(x4) if drop_prob else x4

    # Skip connection 2.
    outputs = layers.Add()([x2, x4])

    return keras.Model(encoded_patches, outputs, name=name)



이제 방금 개발한 요소 위에 `ViTClassifier` 클래스를 구현하겠습니다. 여기서는 ViT 논문에서 사용된 원래 풀링 방법을 그대로 따를 것입니다 - 클래스 토큰을 사용하고 분류에 해당하는 feature representation을 사용합니다.

In [ ]:

class ViTClassifier(keras.Model):
    """Vision Transformer base class."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # Patchify + linear projection + reshaping.
        self.projection = keras.Sequential(
            [
                layers.Conv2D(
                    filters=PROJECTION_DIM,
                    kernel_size=(PATCH_SIZE, PATCH_SIZE),
                    strides=(PATCH_SIZE, PATCH_SIZE),
                    padding="VALID",
                    name="conv_projection",
                ),
                layers.Reshape(
                    target_shape=(NUM_PATCHES, PROJECTION_DIM),
                    name="flatten_projection",
                ),
            ],
            name="projection",
        )

        # Positional embedding.
        init_shape = (
            1,
            NUM_PATCHES + 1,
            PROJECTION_DIM,
        )
        self.positional_embedding = tf.Variable(
            tf.zeros(init_shape), name="pos_embedding"
        )

        # Transformer blocks.
        dpr = [x for x in tf.linspace(0.0, DROP_PATH_RATE, NUM_LAYERS)]
        self.transformer_blocks = [
            transformer(drop_prob=dpr[i], name=f"transformer_block_{i}")
            for i in range(NUM_LAYERS)
        ]

        # CLS token.
        initial_value = tf.zeros((1, 1, PROJECTION_DIM))
        self.cls_token = tf.Variable(
            initial_value=initial_value, trainable=True, name="cls"
        )

        # Other layers.
        self.dropout = layers.Dropout(DROPOUT_RATE)
        self.layer_norm = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)
        self.head = layers.Dense(
            NUM_CLASSES,
            name="classification_head",
        )

    def call(self, inputs, training=True):
        n = tf.shape(inputs)[0]

        # 패치를 만들고 projection 시킨다
        projected_patches = self.projection(inputs)

        # 필요하다면 projected_patches 텐서 뒤에 클래스 토큰을 붙인다
        cls_token = tf.tile(self.cls_token, (n, 1, 1))
        cls_token = tf.cast(cls_token, projected_patches.dtype)
        projected_patches = tf.concat([cls_token, projected_patches], axis=1)

        # positional embeddings과 projected patches 합치기
        encoded_patches = (
            self.positional_embedding + projected_patches
        )  # (B, number_patches, projection_dim)
        encoded_patches = self.dropout(encoded_patches)

        # Transformer 블록을 쌓습니다.
        for transformer_module in self.transformer_blocks:
            # Add a Transformer block.
            encoded_patches = transformer_module(encoded_patches)

        # Final layer normalization.
        representation = self.layer_norm(encoded_patches)

        # Pool representation.
        encoded_patches = representation[:, 0]

        # Classification head.
        output = self.head(encoded_patches)
        return output



이 ViTClassifier 클래스는 ViT를 단독으로 사용할 수 있게 만들었으며 end-to-end 학습이 가능합니다. 이제 DeiT로 확장해 보겠습니다. 다음 그림은 DeiT의 개략도(DeiT 논문에서 가져왔습니다)를 보여줍니다:

![](https://i.imgur.com/5lmg2Xs.png)


클래스 토큰 외에도 DeiT는 distillation을 위한 또 다른 토큰을 가지고 있습니다. distillation 과정에서 클래스 토큰에 해당하는 로짓은 실제 레이블과 비교되고, distillation 토큰에 해당하는 로짓은 Teacher의 예측과 비교됩니다.

In [ ]:
# Student 모델
class ViTDistilled(ViTClassifier):
    def __init__(self, regular_training=False, **kwargs):
        super().__init__(**kwargs)
        self.num_tokens = 2
        self.regular_training = regular_training

        # CLS와 distillation 토큰, 그리고 positional embedding.
        init_value = tf.zeros((1, 1, PROJECTION_DIM))
        self.dist_token = tf.Variable(init_value, name="dist_token")
        self.positional_embedding = tf.Variable(
            tf.zeros(
                (
                    1,
                    NUM_PATCHES + self.num_tokens,
                    PROJECTION_DIM,
                )
            ),
            name="pos_embedding",
        )

        # Head layers.
        self.head = layers.Dense(
            NUM_CLASSES,
            name="classification_head",
        )
        self.head_dist = layers.Dense(
            NUM_CLASSES,
            name="distillation_head",
        )

    def call(self, inputs, training=True):
        n = tf.shape(inputs)[0]

        # 패치를 만들고 projection 시킨다
        projected_patches = self.projection(inputs)

        # 텐서 뒤에 토큰을 붙인다
        cls_token = tf.tile(self.cls_token, (n, 1, 1))
        dist_token = tf.tile(self.dist_token, (n, 1, 1))
        cls_token = tf.cast(cls_token, projected_patches.dtype)
        dist_token = tf.cast(dist_token, projected_patches.dtype)
        projected_patches = tf.concat(
            [cls_token, dist_token, projected_patches], axis=1
        )

        # positional embeddings과 projected patches 합치기
        encoded_patches = (
            self.positional_embedding + projected_patches
        )  # (B, number_patches, projection_dim)
        encoded_patches = self.dropout(encoded_patches)

        # Transformer 블록을 쌓습니다.
        for transformer_module in self.transformer_blocks:
            # Add a Transformer block.
            encoded_patches = transformer_module(encoded_patches)

        # Final layer normalization.
        representation = self.layer_norm(encoded_patches)

        # Classification heads.
        x, x_dist = (
            self.head(representation[:, 0]),
            self.head_dist(representation[:, 1]),
        )

        if not training or self.regular_training:
            # During standard train / finetune, inference average the classifier
            # predictions.
            return (x + x_dist) / 2

        elif training:
            # Only return separate classification predictions when training in distilled
            # mode.
            return x, x_dist



`ViTDistilled` 클래스가 우리의 예상대로 초기화 및 호출될 수 있는지 확인합니다.

In [ ]:
deit_tiny_distilled = ViTDistilled()

dummy_inputs = tf.ones((2, 224, 224, 3))
outputs = deit_tiny_distilled(dummy_inputs, training=False)
print(outputs.shape)

(2, 5)


## Trainer 구현하기

knowledge distillation
([Hinton et al.](https://arxiv.org/abs/1503.02531))에서 일어나는 것과 달리,
KL divergence 뿐만 아니라 temperature-scaled softmax가 사용되는 경우, DeiT 저자들은 다음과 같은 손실 함수를 사용했습니다.

![](https://i.imgur.com/bXdxsBq.png)


여기를 보세요,

* CE is cross-entropy
* `psi` is the softmax function
* Z_s denotes student predictions
* y denotes true labels
* y_t denotes teacher predictions

In [ ]:

# knowledge distillation 하는 class
class DeiT(keras.Model):
    # Reference:
    # https://keras.io/examples/vision/knowledge_distillation/
    def __init__(self, student, teacher, **kwargs):
        super().__init__(**kwargs)
        self.student = student
        self.teacher = teacher

        self.student_loss_tracker = keras.metrics.Mean(name="student_loss")
        self.dist_loss_tracker = keras.metrics.Mean(name="distillation_loss")

    @property
    def metrics(self):
        metrics = super().metrics
        metrics.append(self.student_loss_tracker)
        metrics.append(self.dist_loss_tracker)
        return metrics

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def train_step(self, data):
        # 데이터 풀기
        x, y = data

        # Teacher의 Forward pass
        teacher_predictions = tf.nn.softmax(self.teacher(x, training=False), -1)
        teacher_predictions = tf.argmax(teacher_predictions, -1)

        with tf.GradientTape() as tape:
            # Studentdml Forward pass
            cls_predictions, dist_predictions = self.student(x / 255.0, training=True)

            # student_loss와 distillation_loss 계산
            student_loss = self.student_loss_fn(y, cls_predictions)
            distillation_loss = self.distillation_loss_fn(
                teacher_predictions, dist_predictions
            )
            loss = (student_loss + distillation_loss) / 2

        # Gradients 계산
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Weights 업데이트
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # `compile()`에 명시된 metrics 업데이트
        student_predictions = (cls_predictions + dist_predictions) / 2
        self.compiled_metrics.update_state(y, student_predictions)
        self.dist_loss_tracker.update_state(distillation_loss)
        self.student_loss_tracker.update_state(student_loss)

        # 결과값 반환
        results = {m.name: m.result() for m in self.metrics}
        return results

    def test_step(self, data):
        # 데이터 풀기
        x, y = data

        # x 값 대한 결과값 예측
        y_prediction = self.student(x / 255.0, training=False)

        # loss 계산
        student_loss = self.student_loss_fn(y, y_prediction)

        # metrics 업데이트
        self.compiled_metrics.update_state(y, y_prediction)
        self.student_loss_tracker.update_state(student_loss)

        # Return a dict of performance.
        results = {m.name: m.result() for m in self.metrics}
        return results

    def call(self, inputs):
        return self.student(inputs / 255.0, training=False)


## Teacher 모델 로드하기

이 모델은 BiT family of ResNets
([Kolesnikov et al.](https://arxiv.org/abs/1912.11370))에 기반했고 `tf_flowers`으로 fine-tuned 했습니다. 이 [this notebook](https://github.com/sayakpaul/deit-tf/blob/main/notebooks/bit-teacher.ipynb)을 참조하여 학습이 어떻게 되었는지 알 수 있습니다. Teacher 모델은 Student보다 **약 40배** 많은 약 2억 2천만 개의 파라미터를 가지고 있습니다.

In [ ]:
!wget -q https://github.com/sayakpaul/deit-tf/releases/download/v0.1.0/bit_teacher_flowers.zip
!unzip -q bit_teacher_flowers.zip

In [ ]:
bit_teacher_flowers = keras.models.load_model("bit_teacher_flowers")

## Distillation으로 Student 모델 학습하기

In [ ]:
deit_tiny = ViTDistilled()
deit_distiller = DeiT(student=deit_tiny, teacher=bit_teacher_flowers)

lr_scaled = (BASE_LR / 512) * BATCH_SIZE
deit_distiller.compile(
    optimizer=tfa.optimizers.AdamW(weight_decay=WEIGHT_DECAY, learning_rate=lr_scaled),
    metrics=["accuracy"],
    student_loss_fn=keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1
    ),
    distillation_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
_ = deit_distiller.fit(train_dataset, validation_data=val_dataset, epochs=NUM_EPOCHS)

Epoch 1/20


만약 우리가 똑같은 하이퍼파라미터를 가지고 스크래치로 같은 모델( `ViTClassifier`)을 훈련했다면, 그 모델은 약 59%의 정확도를 얻었을 것입니다. 그 결과를 재현하기 위해 아래와 같이 코드를 바꿔보면 됩니다 :
```
vit_tiny = ViTClassifier()

inputs = keras.Input((RESOLUTION, RESOLUTION, 3))
x = keras.layers.Rescaling(scale=1./255)(inputs)
outputs = deit_tiny(x)
model = keras.Model(inputs, outputs)

model.compile(...)
model.fit(...)
```

## 노트

* Distillation 사용을 통해 CNN 기반 Teacher 모델의 inductive biases을 효과적으로 전달하고 있습니다.
* 흥미롭게도, 이 distillation 전략은 논문에서 보여지는 teacher CNN을 사용했을 때 모델로 트랜스포머보다 더 잘 작동합니다.
* DeiT 모델을 훈련시키기 위해 정규화(regularization)를 사용하는 것은 매우 중요합니다.
* ViT 모델은 truncated normal, random normal, Glorot uniform 등을 포함한 다양한 이니셜라이저의 조합으로 초기화됩니다. 원래 결과의 end-to-end 복제를 원하는 경우 ViT를 주의해서 초기화해야 합니다.
* Fine-tuning을 위해 TensorFlow와 Keras로 pre-trained된 DeiT 모델을 찾으려면 [TF-Hub](https://tfhub.dev/sayakpaul/collections/deit/1)을 사용하면 됩니다.

## 감사의 말

* Ross Wightman은 [`timm`](https://github.com/rwightman/pytorch-image-models)을 최신 상태로 유지해줍니다. TensorFlow로 구현하면서 그의 ViT와 DeiT의 구현을 많이 참고했습니다.
* 다른 프로젝트에서 `ViTClassifier의 일부를 [Aritra Roy Gosthipaty](https://github.com/ariG23498)가 구현했습니다.

* [Google Developers Experts](https://developers.google.com/programs/experts/) 프로그램은 이 예에 대한 실험을 실행하는 데 사용된 GCP 크레딧으로 저를 지원해주는 프로그램입니다.

Example available on HuggingFace:

| Trained Model | Demo |
| :--: | :--: |
| [![Generic badge](https://img.shields.io/badge/🤗%20Model-DEIT-black.svg)](https://huggingface.co/keras-io/deit) | [![Generic badge](https://img.shields.io/badge/🤗%20Spaces-DEIT-black.svg)](https://huggingface.co/spaces/keras-io/deit/) |